In [1]:
# Import necessary pachage; 
import pandas as pd 
import snowflake.connector
from sqlalchemy import create_engine

In [3]:
# Create connect from Snowflake;
engine = create_engine(
    'snowflake://{user}:{password}@{account_identifier}/{database}/{schema}?warehouse={warehouse}&role={role}'.format(
        user='user_name',
        password='user_password',
        account_identifier='account_identifier',
        database='database_name',
        schema='schema_name',
        warehouse='warehouse_name',
        role='role_name'
    )
)

In [5]:
# Queary that return cuurrency exchange :
#      If stocks buy in weekday(excluding Monday) -> the exchange rate is tahken (-1 day) NBP Table_A;
#      If stocks buy in Monday -> the exchange rate is tahken for Friday NBP Table_A
query_conversions="""WITH cte_convertion AS (
        SELECT time::date as stock_date,
            currency_conversion_to_amount,
            currency_conversion_from_amount,
            currency_currency_conversion_from_amount,
            currency_currency_conversion_to_amount,
            currency_conversion_fee,
            currency_currency_conversion_fee
        FROM stocks 
        WHERE action ='Currency conversion'  AND currency_currency_conversion_to_amount='USD'
    ), cte_currency as(
        SELECT  TO_DATE(TO_VARCHAR(data), 'YYYYMMDD') AS currency_date,
                usd
        FROM currenc_rates
    )
    
    
    SELECT  s.stock_date,
            s.currency_conversion_to_amount as usd_bought,
            s.currency_currency_conversion_to_amount,
            (s.currency_conversion_to_amount+COAlESCE(currency_conversion_fee,0)) as usd_netto,
            CASE 
                WHEN s.currency_currency_conversion_to_amount='USD' THEN (s.currency_conversion_to_amount+COALESCE(currency_conversion_fee,0))* c.usd
            END pln_netto,
            s.currency_conversion_fee,
            s.currency_currency_conversion_fee,
            c.usd as exchange_usd_NBP
    FROM cte_convertion as s
    LEFT JOIN cte_currency AS c
        ON s.stock_date = CASE
                         WHEN DAYOFWEEK(s.stock_date) = 1 THEN DATEADD(DAY, 3, c.currency_date)
                         WHEN DAYOFWEEK(s.stock_date) =  0 THEN DATEADD(DAY, 2, c.currency_date)
                         WHEN s.stock_date = '2024-12-27' THEN DATEADD(DAY, 3, c.currency_date) 
                         ELSE DATEADD(DAY, 1, c.currency_date) 
                      END"""

In [7]:
# Execute written above query;
df_convertions = pd.read_sql(query_conversions, engine)

In [9]:
# Change  'stock_date' format to 'date';
df_convertions.loc[:,'stock_date']=pd.to_datetime(df_convertions['stock_date']).dt.date

In [11]:
# Choose only needed columns;
df_convertions=df_convertions[['stock_date','usd_bought','usd_netto','pln_netto','exchange_usd_nbp']]

In [13]:
# Check created DataFrame;
df_convertions

,stock_date,usd_bought,usd_netto,pln_netto,exchange_usd_nbp
0,2024-06-06,25.43,25.39,100.562173,3.9607
1,2024-06-06,25.48,25.44,100.760208,3.9607
2,2024-07-05,25.22,25.18,100.176112,3.9784
3,2024-07-06,25.25,25.21,99.783701,3.9581
4,2024-07-08,25.36,25.32,100.219092,3.9581
5,2024-07-08,25.39,25.35,100.337835,3.9581
6,2024-07-10,25.39,25.35,99.856185,3.9391
7,2024-07-13,1.53,1.53,5.982147,3.9099
8,2024-08-04,50.77,50.69,201.097368,3.9672
9,2024-08-05,51.28,51.20,203.120640,3.9672


In [15]:
# Query to return 'Market buy' and 'Limit buy' records, and only necessary column;
stock_buy_query="""SELECT action,
        time,
        ticker,
        no_of_shares,
        price_per_share,
        exchange_rate,
        currency_result,
        total,
        currency_total
FROM stocks
WHERE action in ('Market buy', 'Limit buy') AND currency_result='USD';"""

In [17]:
# Execute query;
df_stock_buy=pd.read_sql(stock_buy_query, engine)

In [18]:
# Change  'time' column's format to 'date';
df_stock_buy['time']=df_stock_buy['time'].dt.date

In [21]:
# Review created DataFrame;
df_stock_buy

,action,time,ticker,no_of_shares,price_per_share,exchange_rate,currency_result,total,currency_total
0,Market buy,2024-06-17,PYPL,0.334167,59.85,1.00000000,USD,20.0,USD
1,Market buy,2024-06-25,PYPL,0.336812,59.38,1.00000000,USD,20.0,USD
2,Market buy,2024-07-10,V,0.114859,261.19,1.00000000,USD,30.0,USD
3,Market buy,2024-07-10,PDD,0.216794,138.38,1.00000000,USD,30.0,USD
4,Market buy,2024-07-22,META,0.053268,488.10,1.00000000,USD,26.0,USD
5,Market buy,2024-07-22,GOOG,0.135972,183.86,1.00000000,USD,25.0,USD
6,Market buy,2024-08-05,VUAA,0.511298,97.79,1.00000000,USD,50.0,USD
7,Market buy,2024-08-05,GOOG,0.128771,163.08,1.00000000,USD,21.0,USD
8,Market buy,2024-08-05,GOOG,0.157699,164.87,1.00000000,USD,26.0,USD
9,Market buy,2024-08-05,CRWD,0.108381,221.44,1.00000000,USD,24.0,USD


In [39]:
# formation of purchase queue (FIFO);
fifo_queue=[]
for _, row in df_convertions.iterrows():
    fifo_queue.append({
        "usd_left": row["usd_netto"],
        "pln_left": row["pln_netto"],
        "rate": row['exchange_usd_nbp']
    })

In [41]:
# Review created list of dictionaris;
fifo_queue

[{'usd_left': 25.39, 'pln_left': 100.562173, 'rate': 3.9607},
 {'usd_left': 25.44, 'pln_left': 100.760208, 'rate': 3.9607},
 {'usd_left': 25.18, 'pln_left': 100.176112, 'rate': 3.9784},
 {'usd_left': 25.21, 'pln_left': 99.783701, 'rate': 3.9581},
 {'usd_left': 25.32, 'pln_left': 100.219092, 'rate': 3.9581},
 {'usd_left': 25.35, 'pln_left': 100.337835, 'rate': 3.9581},
 {'usd_left': 25.35, 'pln_left': 99.856185, 'rate': 3.9391},
 {'usd_left': 1.53, 'pln_left': 5.982147, 'rate': 3.9099},
 {'usd_left': 50.69, 'pln_left': 201.097368, 'rate': 3.9672},
 {'usd_left': 51.2, 'pln_left': 203.12064, 'rate': 3.9672},
 {'usd_left': 77.28, 'pln_left': 298.949952, 'rate': 3.8684}]

In [43]:
# Processing each stock purchase operation that requires USD;
results = []

for _, buy in df_stock_buy.iterrows():
    usd_needed = buy["total"]
    pln_used = 0

    while usd_needed > 0 and fifo_queue:
        batch = fifo_queue[0]
        take_usd = min(usd_needed, batch["usd_left"])
        take_pln = take_usd * batch["rate"]

        
        batch["usd_left"] -= take_usd
        batch["pln_left"] -= take_pln
        usd_needed -= take_usd
        pln_used += take_pln

        if batch["usd_left"] == 0:
            fifo_queue.pop(0)

    results.append({
        "buy_date": buy["time"],
        "ticker":buy['ticker'],
        'no_of_shares':buy['no_of_shares'],
        'price_per_share':buy['price_per_share'],
        "usd_spent": buy["total"],
        "pln_equivalent": pln_used
    })

# -----------------------
# 4. Result as DataFrame
# -----------------------
df_results = pd.DataFrame(results)
df_results



,buy_date,ticker,no_of_shares,price_per_share,usd_spent,pln_equivalent
0,2024-06-17,PYPL,0.334167,59.85,20.0,79.214000
1,2024-06-25,PYPL,0.336812,59.38,20.0,79.214000
2,2024-07-10,V,0.114859,261.19,30.0,119.160309
3,2024-07-10,PDD,0.216794,138.38,30.0,118.865003
4,2024-07-22,META,0.053268,488.10,26.0,102.910600
5,2024-07-22,GOOG,0.135972,183.86,25.0,98.952500
6,2024-08-05,VUAA,0.511298,97.79,50.0,197.551897
7,2024-08-05,GOOG,0.128771,163.08,21.0,83.311200
8,2024-08-05,GOOG,0.157699,164.87,26.0,103.147200
9,2024-08-05,CRWD,0.108381,221.44,24.0,95.212800


In [45]:
df_results.to_sql("stock_buy_in_usd", con=engine, if_exists="replace", index=False)

10